In [18]:
# -*- coding: utf-8 -*-
from gemm_tiling import gemm_tiling_input_stationary, gemm_tiling_weight_stationary
import transformer_block as tbk
import arch_execution as arch
from util import *
import math
from mapper import *

In [19]:
llm_config = load_config("./input/transformer/input0.json")
model = tbk.Llama_block(llm_config)
tx8_config = load_config('hardware_parameter.json')
hardware = arch.Tx8(tx8_config)
Layers = model.config['L']
ops = model.ops
mapping_result = {}

In [20]:
# =============== FNN Up  ===================
# FFN Gate 与 FFN Up是完全同样的size的计算，这里不列出了
M, K, N = 4096, 4096, 11008
details = False


B = 16
tile_m = 4
tile_n = 86
print(f"FFN Up analysis, B={B}")
utilization = gemm_tiling_weight_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(f"FFN Up, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: input, utilization={utilization:.2f}%")


# [N/tile_n, B*M/tile_m] -- SRAM exceed -- debug shape info
Tm_Tn = [int(tile_m), int(tile_n)]
input_stationary = False  # ishape <--> oshape, internal exchange shape

mapping_result['FFNup'] = gemm_auto_opt_mapper(
    ops['FFN2'], hardware, input_stationary=input_stationary, Tm_Tn=Tm_Tn, details=details)
utilization = mapping_result['FFN2']["utilization"]*100
stationary = "input" if input_stationary else "weight"
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"{stationary} Stationary, Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")

FFN Up analysis, B=16
FFN Up, M=4096, K=4096, N=11008, B=16, tile_m=4, tile_n=86, stationary: input, utilization=98.28%


KeyError: 'FFN2'

In [ ]:
# =============== FNN down  ===================
M, K, N = 4096, 11008, 4096
B = 1


tile_m, tile_n = 128, 4
utilization = gemm_tiling_input_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(f"FFN Down, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: input, utilization={utilization:.2f}%")


# [N/tile_n, B*M/tile_m] -- SRAM exceed -- debug shape info
Tm_Tn = [int(tile_m), int(tile_n)]
input_stationary = True  # ishape <--> oshape, internal exchange shape

mapping_result['FFN2'] = gemm_auto_opt_mapper(
    ops['FFN2'], hardware, input_stationary=input_stationary, Tm_Tn=Tm_Tn, details=details)
utilization = mapping_result['FFN2']["utilization"]*100
stationary = "input" if input_stationary else "weight"
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"{stationary} Stationary, Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")

B = 16
tile_m, tile_n = 4, 128
utilization = gemm_tiling_weight_stationary(
    B, M, K, N, tile_m, tile_n, print_details=False)
print(f"FFN Down, M={M}, K={K}, N={N}, B={B}, tile_m={tile_m}, tile_n={tile_n}, stationary: weight, utilization={utilization:.2f}%")

# [N/tile_n, B*M/tile_m] -- SRAM exceed -- debug shape info
Tm_Tn = [int(tile_m), int(tile_n)]
input_stationary = False  # ishape <--> oshape, internal exchange shape

mapping_result['FFN2'] = gemm_auto_opt_mapper(
    ops['FFN2'], hardware, input_stationary=input_stationary, Tm_Tn=Tm_Tn, details=details)
utilization = mapping_result['FFN2']["utilization"]*100
stationary = "input" if input_stationary else "weight"
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
print(f"{stationary} Stationary, Tm_Tn={Tm_Tn}, Hardware utilization: {utilization:.2f}%")
print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")

FFN Down, M=4096, K=11008, N=4096, B=1, tile_m=128, tile_n=4, stationary: input, utilization=77.63%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
input Stationary, Tm_Tn=[128, 4], Hardware utilization: 90.10%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
FFN Down, M=4096, K=11008, N=4096, B=16, tile_m=4, tile_n=128, stationary: weight, utilization=98.23%
输入不复用则SRAM满足要求
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
weight Stationary, Tm_Tn=[4, 128], Hardware utilization: 77.20%
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
